# 3. Depth Map with Tuning Bar (Aravis port)

Streams live rectified frames from both sensors, computes a disparity map
using `cv2.StereoBM`, and displays an interactive tuning bar to adjust
StereoBM parameters in real time.

Requires calibration files in `calib_result/` produced by
`2.Calibration.ipynb`.

**Controls**
- Drag trackbars to tune StereoBM parameters live
- Set **Load Settings** trackbar → 1 to restore from `3dmap_set.txt`
- Set **Save Settings** trackbar → 1 to write current settings to `3dmap_set.txt`
- Click on the disparity window to read the disparity value at that pixel
- Press **`q`** to quit

In [ ]:
%pip install -r requirements.txt -q

In [ ]:
import gi
gi.require_version('Aravis', '0.8')
from gi.repository import Aravis

import cv2
import os
import numpy as np
import json

## Configuration

In [ ]:
# IMX273 with 2:1 average sensor binning → 720×540 per sensor.
# Combined dual-head frame: 1440×540 (columns interleaved).
DUAL_WIDTH  = 1440
DUAL_HEIGHT = 540

# Size each sub-image is normalised to before rectification and matching.
# Matches the native binned resolution — no downsampling needed.
PROC_SIZE = (720, 540)  # (width, height)

## Helper functions

In [ ]:
def adjust_gamma(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array(
        [((i / 255.0) ** inv_gamma) * 255 for i in range(256)]
    ).astype(np.uint8)
    return cv2.LUT(image, table)


def stereo_depth_map(rectified_pair, variable_mapping):
    sbm = cv2.StereoBM_create(numDisparities=16, blockSize=variable_mapping["SWS"])
    sbm.setPreFilterType(1)
    sbm.setPreFilterSize(variable_mapping["PreFiltSize"])
    sbm.setPreFilterCap(variable_mapping["PreFiltCap"])
    sbm.setSpeckleRange(variable_mapping["SpeckleRange"])
    sbm.setSpeckleWindowSize(variable_mapping["SpeckleSize"])
    sbm.setMinDisparity(variable_mapping["MinDisp"])
    sbm.setNumDisparities(variable_mapping["NumofDisp"])
    sbm.setTextureThreshold(variable_mapping["TxtrThrshld"])
    sbm.setUniquenessRatio(variable_mapping["UniqRatio"])

    disparity            = sbm.compute(rectified_pair[0], rectified_pair[1])
    disparity_normalized = cv2.normalize(disparity, None, 0, 255, cv2.NORM_MINMAX)
    image                = np.array(disparity_normalized, dtype=np.uint8)
    disparity_color      = cv2.applyColorMap(image, cv2.COLORMAP_JET)
    return disparity_color, disparity_normalized


def save_load_map_settings(current_save, current_load, variable_mapping):
    global loading
    settings_file = os.path.join(os.getcwd(), "3dmap_set.txt")

    if current_save != 0:
        result = json.dumps(
            {
                "SADWindowSize":       variable_mapping["SWS"],
                "preFilterSize":       variable_mapping["PreFiltSize"],
                "preFilterCap":        variable_mapping["PreFiltCap"],
                "minDisparity":        variable_mapping["MinDisp"],
                "numberOfDisparities": variable_mapping["NumofDisp"],
                "textureThreshold":    variable_mapping["TxtrThrshld"],
                "uniquenessRatio":     variable_mapping["UniqRatio"],
                "speckleRange":        variable_mapping["SpeckleRange"],
                "speckleWindowSize":   variable_mapping["SpeckleSize"],
            },
            sort_keys=True, indent=4, separators=(",", ": ")
        )
        with open(settings_file, "w") as f:
            f.write(result)
        print(f"Settings saved to {settings_file}")

    if current_load != 0:
        if not os.path.exists(settings_file):
            print(f"Settings file not found: {settings_file}")
            return
        loading = True
        with open(settings_file, "r") as f:
            data = json.load(f)
        cv2.setTrackbarPos("SWS",         "Stereo", data["SADWindowSize"])
        cv2.setTrackbarPos("PreFiltSize",  "Stereo", data["preFilterSize"])
        cv2.setTrackbarPos("PreFiltCap",   "Stereo", data["preFilterCap"])
        cv2.setTrackbarPos("MinDisp",      "Stereo", data["minDisparity"] + 100)
        cv2.setTrackbarPos("NumofDisp",    "Stereo", int(data["numberOfDisparities"] / 16))
        cv2.setTrackbarPos("TxtrThrshld",  "Stereo", data["textureThreshold"])
        cv2.setTrackbarPos("UniqRatio",    "Stereo", data["uniquenessRatio"])
        cv2.setTrackbarPos("SpeckleRange", "Stereo", data["speckleRange"])
        cv2.setTrackbarPos("SpeckleSize",  "Stereo", data["speckleWindowSize"])
        print(f"Parameters loaded from {settings_file}")


def activateTrackbars(x):
    global loading
    loading = False


def create_trackbars():
    # SWS upper limit must not exceed image width or height.
    cv2.createTrackbar("SWS",          "Stereo", 115, 230, activateTrackbars)
    cv2.createTrackbar("SpeckleSize",  "Stereo",   0, 300, activateTrackbars)
    cv2.createTrackbar("SpeckleRange", "Stereo",   0,  40, activateTrackbars)
    cv2.createTrackbar("UniqRatio",    "Stereo",   1,  20, activateTrackbars)
    cv2.createTrackbar("TxtrThrshld",  "Stereo",   0, 1000, activateTrackbars)
    cv2.createTrackbar("NumofDisp",    "Stereo",   1,  16, activateTrackbars)
    cv2.createTrackbar("MinDisp",      "Stereo", 100, 200, activateTrackbars)  # offset by 100 (range -100..100)
    cv2.createTrackbar("PreFiltCap",   "Stereo",   1,  63, activateTrackbars)
    cv2.createTrackbar("PreFiltSize",  "Stereo",   5, 255, activateTrackbars)
    cv2.createTrackbar("Save Settings", "Stereo",  0,   1, activateTrackbars)
    cv2.createTrackbar("Load Settings", "Stereo",  0,   1, activateTrackbars)


def onMouse(event, x, y, flag, disparity_normalized):
    """Print the raw disparity value at the clicked pixel.
    To convert to metric depth: depth = (f_px * baseline_mm) / disparity
    where f_px is focal length in pixels from the calibration intrinsics.
    """
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Disparity at ({x}, {y}): {disparity_normalized[y][x]:.1f}")

## Recommended StereoBM starting parameters

| Parameter | Recommended |
|-----------|-------------|
| SAD Window Size | 19 |
| Speckle Size | 14 |
| Speckle Range | 27 |
| Uniqueness Ratio | 8 |
| Texture Threshold | 1000 |
| Number of Disparities | 16 |
| Minimum Disparity | 40 |
| Pre Filter Cap | 63 |
| Pre Filter Size | 9 |

These are written to / loaded from `3dmap_set.txt` via the trackbar.

## Device discovery

In [ ]:
Aravis.update_device_list()
n_devices = Aravis.get_n_devices()

if n_devices == 0:
    raise RuntimeError(
        "No GigE Vision camera found. "
        "Check network connection and firewall settings."
    )

print(f"Found {n_devices} device(s):")
for i in range(n_devices):
    print(f"  [{i}] {Aravis.get_device_id(i)}")

camera = Aravis.Camera.new(Aravis.get_device_id(0))
device = camera.get_device()
print(f"\nConnected to: {camera.get_model_name()}")

## Camera configuration

In [ ]:
camera.gv_auto_packet_size()

device.set_string_feature_value("ImagerOutputSelector", "All")

# 2:1 average sensor binning — see README for rationale.
device.set_string_feature_value("BinningSelector",       "Sensor")
device.set_integer_feature_value("BinningVertical",      2)
device.set_integer_feature_value("BinningHorizontal",    2)
device.set_string_feature_value("BinningVerticalMode",   "Average")
device.set_string_feature_value("BinningHorizontalMode", "Average")

device.set_integer_feature_value("Width",  DUAL_WIDTH)
device.set_integer_feature_value("Height", DUAL_HEIGHT)

isp_bayer_pattern = device.get_string_feature_value("IspBayerPattern")
if isp_bayer_pattern != "NONE":
    device.set_string_feature_value("PixelFormat", "DualBayerRG8")
else:
    device.set_string_feature_value("PixelFormat", "DualMono8")

print(f"IspBayerPattern : {isp_bayer_pattern}")
print(f"PixelFormat     : {device.get_string_feature_value('PixelFormat')}")
print(f"Frame size      : {device.get_integer_feature_value('Width')} × "
      f"{device.get_integer_feature_value('Height')}")

## Live depth map loop

Drag trackbars to tune StereoBM parameters.  
Press **`q`** to quit.

In [ ]:
# Load stereo calibration maps once — produced by 2.Calibration.ipynb.
calib_result_path = os.path.join(os.getcwd(), "calib_result")
if not os.path.isdir(calib_result_path):
    raise FileNotFoundError(
        f"Calibration folder not found: {calib_result_path}\n"
        "Run 2.Calibration.ipynb first."
    )

map_l1 = np.load(os.path.join(calib_result_path, "undistortion_map_left.npy"))
map_l2 = np.load(os.path.join(calib_result_path, "rectification_map_left.npy"))
map_r1 = np.load(os.path.join(calib_result_path, "undistortion_map_right.npy"))
map_r2 = np.load(os.path.join(calib_result_path, "rectification_map_right.npy"))

# Pre-allocate Aravis stream buffers.
payload = camera.get_payload()
stream  = camera.create_stream(None, None)
for _ in range(5):
    stream.push_buffer(Aravis.Buffer.new_allocate(payload))

# Initialise windows and trackbars.
cv2.namedWindow("Stereo",    cv2.WINDOW_NORMAL)
cv2.namedWindow("Disparity", cv2.WINDOW_NORMAL)
cv2.namedWindow("Frame",     cv2.WINDOW_NORMAL)
create_trackbars()

trackbar_vars = [
    "SWS", "SpeckleSize", "SpeckleRange", "UniqRatio",
    "TxtrThrshld", "NumofDisp", "MinDisp", "PreFiltCap", "PreFiltSize",
]

variable_mapping = {
    "SWS": 15, "SpeckleSize": 100, "SpeckleRange": 15,
    "UniqRatio": 10, "TxtrThrshld": 100, "NumofDisp": 16,
    "MinDisp": -25, "PreFiltCap": 30, "PreFiltSize": 105,
}

loading = False
disparity_normalized = None

print("Streaming. Press 'q' to quit.")
camera.start_acquisition()

try:
    while True:
        buf = stream.timeout_pop_buffer(1_000_000)
        if buf is None:
            print("Timeout waiting for frame.")
            continue

        if buf.get_status() != Aravis.BufferStatus.SUCCESS:
            stream.push_buffer(buf)
            continue

        data   = buf.get_data()
        height = buf.get_image_height()
        width  = buf.get_image_width()
        arr    = np.frombuffer(data, dtype=np.uint8)[: height * width].reshape(height, width)

        # Split interleaved dual-head columns.
        img0 = arr[:, 0::2].copy()  # left  -> 720 x 540
        img1 = arr[:, 1::2].copy()  # right -> 720 x 540

        stream.push_buffer(buf)

        img0 = adjust_gamma(img0, 2.5)
        img1 = adjust_gamma(img1, 2.5)

        # StereoBM requires grayscale input.
        if isp_bayer_pattern != "NONE":
            img0 = cv2.cvtColor(img0, cv2.COLOR_BAYER_BG2GRAY)
            img1 = cv2.cvtColor(img1, cv2.COLOR_BAYER_BG2GRAY)

        # Safety resize — images should already be PROC_SIZE after binning.
        img0 = cv2.resize(img0, PROC_SIZE)
        img1 = cv2.resize(img1, PROC_SIZE)

        rect_left  = cv2.remap(img0, map_l1, map_l2, cv2.INTER_LINEAR)
        rect_right = cv2.remap(img1, map_r1, map_r2, cv2.INTER_LINEAR)
        rectified_pair = (rect_left, rect_right)

        if not loading:
            # Read and sanitise trackbar values.
            for v in trackbar_vars:
                val = cv2.getTrackbarPos(v, "Stereo")
                if v in ("SWS", "PreFiltSize"):
                    val = max(val, 5)
                    if val % 2 == 0:
                        val += 1
                elif v == "NumofDisp":
                    val = max(val, 1) * 16
                elif v == "MinDisp":
                    val = val - 100
                elif v in ("UniqRatio", "PreFiltCap"):
                    val = max(val, 1)
                variable_mapping[v] = val

            current_save = cv2.getTrackbarPos("Save Settings", "Stereo")
            current_load = cv2.getTrackbarPos("Load Settings", "Stereo")
            save_load_map_settings(current_save, current_load, variable_mapping)
            cv2.setTrackbarPos("Save Settings", "Stereo", 0)
            cv2.setTrackbarPos("Load Settings", "Stereo", 0)

            disparity_color, disparity_normalized = stereo_depth_map(
                rectified_pair, variable_mapping
            )

            if disparity_normalized is not None:
                cv2.setMouseCallback("Stereo", onMouse, disparity_normalized)

            cv2.imshow("Disparity", disparity_color)
            cv2.resizeWindow("Disparity", 720, 540)
            cv2.imshow("Frame", np.hstack((rectified_pair[0], rectified_pair[1])))

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

finally:
    camera.stop_acquisition()
    cv2.destroyAllWindows()
    print("Stream stopped.")